In [63]:
!pip install gpytorch

In [64]:
from google.colab import files

In [65]:
!pip install cocopp
!git clone https://github.com/numbbo/coco.git
!cd coco; python do.py run-python

fatal: destination path 'coco' already exists and is not an empty directory.
AML	['code-experiments/src/coco_random.c', 'code-experiments/src/coco_suite.c', 'code-experiments/src/coco_observer.c', 'code-experiments/src/coco_archive.c', 'code-experiments/src/coco_runtime_c.c'] -> code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/build/python/cython/coco.c.in to code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/src/coco.h to code-experiments/build/python/cython/coco.h
COPY	code-experiments/src/bbob2009_testcases.txt -> code-experiments/build/python/bbob2009_testcases.txt
COPY	code-experiments/src/bbob2009_testcases2.txt -> code-experiments/build/python/bbob2009_testcases2.txt
COPY	code-experiments/build/python/README.md -> code-experiments/build/python/README.txt
EXPAND	code-experiments/build/python/setup.py.in to code-experiments/build/python/setup.py
PYTHON	setup.py install in code-experiments/build/python
PYTHON	example_experiment.py bbob in code-ex

In [66]:
!cd coco/code-experiments/build/python; git clone https://github.com/uber-research/TuRBO

fatal: destination path 'TuRBO' already exists and is not an empty directory.


In [74]:
%%writefile simplest_run.py

#!/usr/bin/env python
"""A short and simple example experiment with restarts.

The code is fully functional but mainly emphasises on readability.
Hence produces only rudimentary progress messages and does not provide
batch distribution or timing prints, as `example_experiment2.py` does.

To apply the code to a different solver, `fmin` must be re-assigned or
re-defined accordingly. For example, using `cma.fmin` instead of
`scipy.optimize.fmin` can be done like::

>>> import cma  # doctest:+SKIP
>>> def fmin(fun, x0):
...     return cma.fmin(fun, x0, 2, {'verbose':-9})

"""
from __future__ import division, print_function
import cocoex, cocopp  # experimentation and post-processing modules
import scipy.optimize  # to define the solver to be benchmarked
from TuRBO.turbo import Turbo1
import numpy as np
from numpy.random import rand  # for randomised restarts
import os, webbrowser  # to show post-processed results in the browser

def fmin(problem, x0, verbose = False):
    dim = len(x0)
    print(f'current x0 is {x0}')
    fmin_ = Turbo1(problem,
                    lb = -5 * np.ones(dim),
                    ub = 5 * np.ones(dim),
                    n_init = 2 * dim,
                    max_evals = 200,
                    verbose = True,
                    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
                    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
                    n_training_steps=100,  # Number of steps of ADAM to learn the hypers
                    min_cuda=1024,  # Run on the CPU for small datasets
                    device="cpu",  # "cpu" or "cuda"
                    dtype="float64")
    #fmin_.succtol = 3
    return fmin_.optimize()

### input
suite_name = "bbob"
output_folder = "scipy-optimize-fmin"
#fmin = scipy.optimize.fmin_bfgs
budget_multiplier = 1  # increase to 10, 100, ...

### prepare
suite = cocoex.Suite(suite_name, "instances: 2"
                                , "dimensions: 2 instance_indices:1")
observer = cocoex.Observer(suite_name, "result_folder: " + output_folder)
minimal_print = cocoex.utilities.MiniPrint()

### go
for problem in suite:  # this loop will take several minutes or longer
    problem.observe_with(observer)  # generates the data for cocopp post-processing
    x0 = problem.initial_solution
    # apply restarts while neither the problem is solved nor the budget is exhausted
    while (problem.evaluations < problem.dimension * budget_multiplier
           and not problem.final_target_hit):
        fmin(problem, x0, False)  # here we assume that `fmin` evaluates the final/returned solution
        x0 = problem.lower_bounds + ((rand(problem.dimension) + rand(problem.dimension)) *
                    (problem.upper_bounds - problem.lower_bounds) / 2)
    minimal_print(problem, final=problem.index == len(suite) - 1)

### post-process data
cocopp.main(observer.result_folder)  # re-run folders look like "...-001" etc
webbrowser.open("file://" + os.getcwd() + "/ppdata/index.html")

Writing simplest_run.py


In [75]:
!ls; mv simplest_run.py coco/code-experiments/build/python;

coco  sample_data  simplest_run.py


In [76]:
!cd coco/code-experiments/build/python; python simplest_run.py

COCO INFO: Results will be output to folder exdata/scipy-optimize-fmin-016
current x0 is [0. 0.]
Using dtype = torch.float64 
Using device = cpu
Starting from fbest = 418.6
6) New best: 409.7
8) New best: 405.4
9) New best: 395.4
13) New best: 394.7
17) New best: 394.7
18) New best: 394.6
19) New best: 394.5
28) New best: 394.5
36) New best: 394.5
37) New best: 394.5
41) New best: 394.5
41) Restarting with fbest = 394.5
Starting from fbest = 397.4
79) Restarting with fbest = 394.5
Starting from fbest = 398.9
104) New best: 394.5
117) New best: 394.5
117) Restarting with fbest = 394.5
Starting from fbest = 402.4
156) Restarting with fbest = 394.5
Starting from fbest = 414.1
197) New best: 394.5
197) Restarting with fbest = 394.5
Starting from fbest = 397.6
236) Restarting with fbest = 394.5
Starting from fbest = 404.9
278) Restarting with fbest = 394.5
Starting from fbest = 400.3
314) New best: 394.5
315) Restarting with fbest = 394.5
Starting from fbest = 397.9
353) Restarting with fbe

In [77]:
!cd coco/code-experiments/build/python; zip -r ppdata.zip ppdata

updating: ppdata/ (stored 0%)
updating: ppdata/scipy-optimize-fmin-008/ (stored 0%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim.html (deflated 69%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f002.pdf (deflated 19%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f002.svg (deflated 83%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f003.pdf (deflated 13%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f003.svg (deflated 77%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f008.pdf (deflated 18%)
updating: ppdata/scipy-optimize-fmin-008/pptable_f003_02D.tex (deflated 75%)
updating: ppdata/scipy-optimize-fmin-008/pptable_f017_02D.tex (deflated 75%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f005.svg (deflated 83%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f006.pdf (deflated 19%)
updating: ppdata/scipy-optimize-fmin-008/ppfigdim_f022.pdf (deflated 13%)
updating: ppdata/scipy-optimize-fmin-008/pptable_f007_02D.tex (deflated 76%)
updating: ppdata/scipy-

In [78]:
uploaded = files.download('coco/code-experiments/build/python/ppdata.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>